In [3]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듭니다.
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [ ]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [ ]:
print(train_input.shape, train_target.shape)

In [ ]:
print(test_input.shape, test_target.shape)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 10, figsize=(10,10))
for i in range(10):
    axs[i].imshow(train_input[i], cmap='gray_r')
    axs[i].axis('off')
plt.show()

In [ ]:
print([train_target[i] for i in range(10)])

In [ ]:
import numpy as np

print(np.unique(train_target, return_counts=True))

In [ ]:
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

In [ ]:
print(train_scaled.shape)

모델 선택 & 데이터 전처리
- 훈련 샘플이 60000개나 되기 때문에 한 번에 다 사용하는 것보다 하나씩 꺼내서 훈련하는 것이 더 효율적일듯 -> 손실함수를 로지스틱으로 한 sgd쓰자
- SGDClassfier는 표준화 전처리가 필요했었음 -> 근데 어차피 각 픽셀들이 0~255 사이의 정수니까, 그냥 255로 나눠서 정규화

모델 훈련
- 경사하강법을 활용한 로지스틱 회귀모델을, cross_validate를 훈련&평가함


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDClassifier

sc = SGDClassifier(loss='log_loss', max_iter=5, random_state=42)

scores = cross_validate(sc, train_scaled, train_target, n_jobs=-1)
print(np.mean(scores['test_score']))

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# 인공신경망에서는 데이터 충분해서 교차검증 굳이 안 함
from sklearn.model_selection import train_test_split # 그래서 검증 세트를 수동으로 덜어내서 사용하겠음

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [ ]:
print(train_scaled.shape, train_target.shape)

In [ ]:
print(val_scaled.shape, val_target.shape)

In [ ]:
# 인공신경망의 출력층을 만들어보자
# 가장 기본인 '밀집층 dense'로
dense = keras.layers.Dense(10, activation='softmax', input_shape=(784,))
#유닛 10개/출력에 적용할 함수는 소프트맥스/입력은 784개

In [ ]:
model = keras.Sequential(dense)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy') # 지금까지 썼던 사이킷런과 달리, 케라스 모델은 훈련하기 전에 설정 단계가 따로 있음.
#➡️ .compile() 에서 손실함수, 측정값 등을 설정하면 됨!


loss : 손실함수의 종류 (이거는 필수!!!!⭐)
- 이진분류는 loss='binary_crossentropy' ; 이진 크로스 엔트로피
- 다중분류는 loss='categorical_crossentropy' ; 크로스 엔트로피
- 근데, sparse_categorical_crossentropy❓
원래 다중분류에서 크로스엔트로피 손실함수를 쓰려면 타깃값에 '원-핫 인코딩' 처리를 해줘야 함(➕플러스알파 참조). 근데, 그걸 안 하고 그냥 정수로 된 타깃값을 그냥 사용할 경우에 이 손실함수를 사용함.
metrics : 기본적으로 출력되는 '손실 값' 이외에 추가로 측정하고 싶은 값.

In [ ]:
print(train_target[:10])

In [ ]:
model.fit(train_scaled, train_target, epochs=5)

In [ ]:
model.evaluate(val_scaled, val_target) #케라스에서는 .evaluate() 로 모델 성능 평가함. (🆚 사이킷런은 .score() 였음)

API 방식 비교

사이킷런 vs 케라스

1. 사이킷런 모델
- 모델 객체를 만들면서 왠만한 매개변수 다 지정함 -> fit로 훈련 -> score로 평가

2. 케라스 모델
- 사이킷런보다 모델 만드는 과정이 더 세분화된 느낌(훈련, 평가 과정은 비슷)
- 층 객체를 따로 만들고 -> 그걸 모델에 Sequential로 추가하고 -> 그 모델의 설정도 compile로 따로 해줌 -> fit으로 훈련하는 건 똑같고 -> 평가는 evaluate로 함